In [1]:
from transformers import AutoTokenizer, pipeline, AutoModelForCausalLM
import json
import torch
from transformers import (AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline)

tokenizer = AutoTokenizer.from_pretrained("./models/hf-frompretrained-download/meta-llama/Meta-Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("./models/hf-frompretrained-downloadmeta-llama/Meta-Llama-3-8B-Instruct")




c:\Users\Trung\anaconda3\envs\master_torch_gpu\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.36s/it]


In [2]:
#from transformers import pipeline
pipeline = pipeline("text-generation",
                    model = model,
                    tokenizer=tokenizer, 
                    eos_token_id=tokenizer.eos_token_id,
                    max_length=520,
                    num_return_sequences =1,
                    device_map = "auto",
                    )




In [3]:
def get_response(prompt):
    sequences = pipeline(
        prompt,
        max_length=2048,
        num_return_sequences=1,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id,
        truncation=True,
    )
    gen_text = sequences[0]["generated_text"]
    
    # Remove the input prompt from the generated text
    response = gen_text[len(prompt):].strip()
    
    return response

In [4]:
def create_user_profile(review):
    prompt = f"""
    You are an e-commerce recommender specialist. Your task is to create a comprehensive user profile based on the following reviews, listed in chronological order (oldest to newest):

    {review}

    Analyze this information and create a user profile following these steps:

1. Long-term preferences: Identify themes and valued attributes.
2. Short-term interests: Note recent preferences or emerging interests.
3. Writing style and sentiment: Analyze tone, vocabulary, and overall sentiment.
4. Demographic information: Infer age range, possible gender, and lifestyle.
5. User profile summary: Combine insights into a concise profile.

Present your analysis in a structured format, using clear headings for each section. Do not include any code in your response. Focus on creating a vivid picture of the user's preferences, habits, and potential future interests.The Created User Profile should not exceed 200 words """
    return get_response(prompt)



In [5]:
# Example usage
review = """{"rating": 5.0, "title": "Great for at home use and so easy to use!", "text": "This is perfect for my between salon visits. I have been using this now twice a week for over a month and I absolutely love it! My skin looks amazing and feels super smooth and silky. This is also super easy to use (just follow instructions). I can see already that I will begin expanding the time between visits which will definitely help me save money in the long run. Highly recommend!", "images": [], "asin": "B08P2DZB4X", "parent_asin": "B08P2DZB4X", "user_id": "AFSKPY37N3C43SOI5IEXEK5JSIYA", "timestamp": 1627391044559, "helpful_vote": 0, "verified_purchase": false, "product_name": "NIRA Skincare Laser & Serum Bundle - Includes Anti-Aging Laser & Hyaluronic Acid Serum - Reduces Appearance of Fine Lines & Wrinkles - FDA Cleared"}"""

profile = create_user_profile(review)
print(profile)

**User Profile**

**Long-term Preferences**
The user values effectiveness, ease of use, and convenience when it comes to their skincare routine. They prioritize products that deliver noticeable results and appreciate simplicity.

**Short-term Interests**
Recent purchases suggest the user has become interested in anti-aging treatments and laser-based solutions. Their enthusiasm for NIRA's product bundle indicates they may explore other similar offerings.

**Writing Style and Sentiment**
The reviewer writes enthusiastically, with a strong emphasis on positive experiences. They employ descriptive language to convey satisfaction, indicating a high level of engagement. The text exudes confidence, suggesting the user trusts their own opinions.

**Demographic Information**
Based on writing style and content, we infer:
	* Age Range: Late twenties to early thirties
	* Possible Gender: Female
	* Lifestyle: Busy professional seeking efficient self-care routines

**Summary**: Our user profile reve

In [6]:
profile = f"""**User Profile**
**Long-term Preferences**
The user values effectiveness, ease of use, and convenience when it comes to their skincare routine. They prioritize products that deliver noticeable results and appreciate simplicity.

**Short-term Interests**
Recent purchases suggest the user has become interested in anti-aging treatments and laser-based solutions. Their enthusiasm for NIRA's product bundle indicates they may explore other similar offerings.

**Writing Style and Sentiment**
The reviewer writes enthusiastically, with a strong emphasis on positive experiences. They employ descriptive language to convey satisfaction, indicating a high level of engagement. The text exudes confidence, suggesting the user trusts their own opinions.

**Demographic Information**
Based on writing style and content, we infer:
	* Age Range: Late twenties to early thirties
	* Possible Gender: Female
	* Lifestyle: Busy professional seeking efficient self-care routines

**Summary**: Our user profile reveals a proactive individual prioritizing effective skincare practices. With a focus on convenience and simplicity, she explores innovative solutions like laser technology. Her enthusiastic nature suggests loyalty towards brands offering seamless experience. As our data grows, expect her interests to shift toward more advanced, result-driven products."""

In [38]:
def create_preliminary_recommendations(user_profile):
    print("creatint Prelim rec")
    prompt =f""" You are an expert recommendation system. Based on the following user profile:

{user_profile}

Generate three general product categories that match the user's preferences and interests. For each category, provide:

1. Name: [Non ambiguous Product or category name that is non ambiguous]
   Reason: [1-2 sentences explaining the fit]

**Guidelines:**
- Do not mention specific brands or product names,except if there is any preference mentioned in the User Profile.
- Use plain text with a numbered list.
- Strictly follow the format"""
    return get_response(prompt)

In [39]:
preliminary_rec = create_preliminary_recommendations(profile)
print(preliminary_rec)

creatint Prelim rec
provided above.



Here are your recommendations based on the given information:


1. **Smart Skincare Solutions**
Reason: This category aligns with the user's interest in exploring new technologies, such as laser-based treatments, which promise noticeable results.


2. **Effortless Beauty Essentials**
Reason: Given the user's priority on convenience and simplicity, this category will appeal to them by focusing on easy-to-use, time-efficient beauty essentials that still deliver effective outcomes.


3. **Result-Oriented Regimens**
Reason: Building upon the user's enthusiasm for anti-aging treatments, this category caters to individuals seeking potent regimens that address specific concerns (e.g., fine lines, wrinkles) while providing clear instructions for optimal usage.
